# NEURAL NETWORK AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAHHHH

{'10 SR': 0, '100 SR': 1, '200 SR': 2, '5 SR': 3, '5 SR Poly': 4, '50 SR': 5, '500 SR': 6}

## Code

### Libraries

In [2]:
!pip install torch torchvision
!pip install scikit-learn

In [3]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm
import numpy as np
from sklearn.metrics import classification_report

### Load, Transform Data

In [5]:
path = "C:\\Users\\Abdu\\Desktop\\Images Dataset"
# Define transformations for the images.
transform = transforms.Compose([
    transforms.Resize((256, 256)),       # Resize images to 256x256
    transforms.ToTensor(),               # Convert images to PyTorch tensors
])

# Load the dataset
dataset = datasets.ImageFolder(root=path, transform=transform)
dataset.transform

Compose(
    Resize(size=(256, 256), interpolation=bilinear, max_size=None, antialias=True)
    ToTensor()
)

### Split Data


In [10]:
# Splitting the dataset
torch.manual_seed(42)  # Set the PyTorch seed
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print(f"Test Size: {len(test_dataset)}\nTrain Size: {len(train_dataset)}")

Test Size: 266
Train Size: 1064


In [26]:
class CNN(nn.Module):
    def __init__(self, num_classes):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.pool1 = nn.MaxPool2d(2, 2)
        
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.pool2 = nn.MaxPool2d(2, 2)
        
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.pool3 = nn.MaxPool2d(2, 2)
        
        # Assuming the input size is 256x256, after three pooling layers the size is 256/8 = 32
        # Adjust the number below if the input size is different
        self.fc1 = nn.Linear(128 * 32 * 32, 512)
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.pool1(F.relu(self.bn1(self.conv1(x))))
        x = self.pool2(F.relu(self.bn2(self.conv2(x))))
        x = self.pool3(F.relu(self.bn3(self.conv3(x))))
        
        x = x.view(-1, 128 * 32 * 32)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [28]:
# Assuming model, train_loader, and criterion are already defined
num_classes = len(dataset.classes)
model = CNN(num_classes)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss()

num_epochs = 30
for epoch in range(num_epochs):
    model.train()  # Make sure the model is in training mode which enables dropout etc.
    correct = 0
    total = 0
    running_loss = 0.0

    loop = tqdm(train_loader, leave=True)  # 'leave=True' ensures the final bar stays
    for inputs, labels in loop:
        optimizer.zero_grad()  # Clear gradients
        outputs = model(inputs)  # Get model outputs

        loss = criterion(outputs, labels)  # Compute the loss
        loss.backward()  # Backpropagate the error
        optimizer.step()  # Adjust model weights
        
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        running_loss += loss.item()

        # Calculate accuracy and average loss
        accuracy = 100 * correct / total
        avg_loss = running_loss / (loop.n + 1)  # loop.n gives the current iteration number
        
        # Update the progress bar
        loop.set_description(f"Epoch [{epoch+1}/{num_epochs}]")
        loop.set_postfix(loss=avg_loss, accuracy=f"{accuracy:.2f}%")

Epoch [30/30]: 100%|██████████| 34/34 [01:09<00:00,  2.05s/it, accuracy=99.15%, loss=0.0373]


In [30]:
# Initialize correct and total counters
correct = 0
total = 0

# Lists to collect all predictions and true labels
all_labels = []
all_predictions = []

# Evaluation mode and no gradient calculation
model.eval()
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        predicted_classes = torch.max(outputs, dim=1)[1]
        
        # Accumulate total and correct predictions
        total += labels.size(0)
        correct += (predicted_classes == labels).sum().item()

        # Extend lists for classification report
        all_predictions.extend(predicted_classes.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Calculate accuracy
print(f'The Accuracy of the model is: {(correct / total) * 100:.2f}%')

# Generate and print classification report
class_report = classification_report(all_labels, all_predictions)
print(class_report)


The Accuracy of the model is: 78.20%
              precision    recall  f1-score   support

           0       0.89      0.85      0.87        48
           1       0.74      0.86      0.79        29
           2       0.82      0.86      0.84        43
           3       0.62      0.65      0.63        37
           4       0.58      0.53      0.55        34
           5       0.97      0.90      0.93        39
           6       0.80      0.78      0.79        36

    accuracy                           0.78       266
   macro avg       0.77      0.78      0.77       266
weighted avg       0.78      0.78      0.78       266



In [ ]:
# For the Inputs
"""
input_image = "PATH TO IMAGE"
model.eval()  # Set the model to evaluation mode

outputs = model(input_image)
probabilities = F.softmax(outputs, dim=1)
predicted_class = torch.argmax(probabilities, dim=1)
max_probability = torch.max(probabilities, dim=1)

print("Predicted Class:", predicted_class.item())
print("Max Probability:", max_probability.values.item())

"""